## 搭建神经网络

神经网络由对数据进行操作的layers/modules组成。  
torch.nn的命名空间提供了构建神经网络所需的所有构建模块。  
所有模块都在pytorch的子类nn.Module里。  
一个神经网络本身就是由其他模块（层）组成的模块，这样的嵌套结构允许轻松地构建和管理复杂的体系结构。  

本例中，我们搭建一个神经网络对FashionMNIST数据集中的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device} device')

using cpu device


In [4]:
# 定义网络
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
# 创建NeuralNetwork的一个实例，并将其转移到device中，打印模型结构
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# 向模型中传入数据
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)  # 注意加(dim=1) 列
y_pred = pred_probab.argmax(1)  #最大值
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


## 分层分析

In [7]:
# 以3张图像为minibatch为例子
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# 铺平
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# 线性层是一个模块，它使用其存储的权重和偏差对输入应用线性变换。
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# ReLU
# 非线性激活在模型的输入和输出之间创建复杂的映射。它们应用于线性变换后引入非线性，帮助神经网络学习各种各样的现象。
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0672, -0.3819,  0.4734, -0.1601,  0.4468, -0.0515, -0.1005, -0.2303,
         -0.2208,  0.1420,  0.1256,  0.2804,  0.3876,  0.4575,  0.0914, -0.0508,
         -0.3695, -0.4391,  0.0090,  0.4726],
        [ 0.2146, -0.4304,  0.3262, -0.2682,  0.3640, -0.0693, -0.1019, -0.1205,
         -0.3293,  0.1448,  0.2999,  0.7261, -0.2730,  0.5481,  0.0859,  0.2990,
         -0.4373, -0.5027, -0.2479,  0.5150],
        [-0.0042,  0.0893,  0.4308, -0.4727,  0.2384, -0.0752, -0.1041, -0.0566,
         -0.3260, -0.0075,  0.1022,  0.5125, -0.4894,  0.3087,  0.2494, -0.0634,
          0.0554, -0.6367, -0.1541,  0.3920]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.4734, 0.0000, 0.4468, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1420, 0.1256, 0.2804, 0.3876, 0.4575, 0.0914, 0.0000, 0.0000, 0.0000,
         0.0090, 0.4726],
        [0.2146, 0.0000, 0.3262, 0.0000, 0.3640, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1448, 0.2999, 0.7261, 0.0000, 0.5481, 0.085

In [11]:
# nn.Sequential是模块的有序容器。
# 数据按照定义的顺序传递给所有模块。您可以使用顺序容器将类似seq_modules的快速网络组合在一起。
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
# 神经网络的最后一个线性层返回logits -原始值[-infty, infty] -这些值被传递给nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
# 神经网络内部的许多层都是参数化的，即在训练过程中有相关的权重和偏差被优化。
# 子类nn.Module模块自动跟踪模型对象中定义的所有字段，并使用模型的parameters()或named_parameters()方法访问所有参数。
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0072,  0.0114,  0.0164,  ...,  0.0298, -0.0242,  0.0181],
        [-0.0310, -0.0290,  0.0341,  ...,  0.0167,  0.0087,  0.0139]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0137,  0.0260], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0419,  0.0164, -0.0154,  ..., -0.0149, -0.0316,  0.0308],
        [-0.0328,  0.0026, -0.0370,  ...,  0.0411, -0.0149, -0.0287]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | S